In [1]:
import pandas as pd
import numpy as np

In [2]:
orderbook = pd.read_csv("./orderbook_data.csv")

In [3]:
orderbook = orderbook.drop(["Unnamed: 0","exchange"], axis=1)
orderbook.head()

timestamp    price  quantity  type
0  1554086597  4123.18  1.000000  bids
1  1554086597  4124.04  0.050978  asks
2  1554086597  4123.17  1.000000  bids
3  1554086597  4125.05  2.432000  asks
4  1554086597  4122.86  3.000000  bids

In [4]:
orderbook = pd.get_dummies(orderbook, columns=['type'])

In [5]:
orderbook.dtypes

timestamp      int64
price        float64
quantity     float64
type_asks      uint8
type_bids      uint8
dtype: object

In [31]:
# label = np.array(orderbook["price"])
# features = orderbook.drop(["price"],axis=1)

In [32]:
# feature_list = list(features.columns)

In [33]:
# features = np.array(features)

In [9]:
# from sklearn.model_selection import train_test_split
# train_features, test_features, train_labels, test_labels = train_test_split(features, label, test_size = 0.20, random_state = 42)

In [27]:
# Out of Time Train-Test Split
total_records = len(orderbook)

split = int(total_records * 0.8)
train = orderbook[:split]
test = orderbook[split:]

train_label = np.array(train["price"])
train_features = train.drop(["price"],axis = 1)
test_label = np.array(test["price"])
test_features = test.drop(["price"],axis = 1)

In [28]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (8000, 4)
Training Labels Shape: (8000,)
Testing Features Shape: (2000, 4)
Testing Labels Shape: (2000,)


In [43]:
# SHOULD BE ATLEAST BETTER THAN WHEN PRICE = QUANTITY
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('quantity')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

In [35]:
feature_list

['timestamp', 'quantity', 'type_asks', 'type_bids']

In [36]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [37]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1008.11 degrees.


In [38]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 79.82 %.


In [39]:
from sklearn.metrics import r2_score

r2 = r2_score(test_labels, predictions) 
print ('R2 Score : ', round(r2,4))

R2 Score :  -0.1046


In [40]:
df = pd.DataFrame(data=[test_labels,predictions])

In [41]:
df.head()

0            1            2            3            4     \
0  3999.740000  3803.000000  6600.000000  3794.270000  5051.000000   
1  4616.315356  4845.694526  4210.052296  4820.458691  4388.049787   

          5            6            7            8            9     \
0  3950.000000  3825.560000  4009.320000  4501.100000  4810.010000   
1  4715.720274  5163.170746  4715.720274  4745.877075  4312.337171   

      ...              1990         1991         1992         1993  \
0     ...       4050.000000  3797.310000  3875.100000  3778.530000   
1     ...       4255.780564  4602.766059  4951.887282  4928.402964   

          1994         1995         1996         1997         1998  \
0  6545.200000  3900.100000  4353.000000  7490.000000  4150.000000   
1  4413.458286  4664.520139  4614.382642  5171.009099  4505.035013   

          1999  
0  7978.900000  
1  5217.612878  

[2 rows x 2000 columns]

In [42]:
# Getting more data: We can train the model on more orders (not sure if this would help, but worth a shot), and bring in more features (which can only happen if we can ID the orders - I'll do some more digging on this)
# Better feature engineering: I sense that we could probably derive more information out of our current features. We should try to get a on a quick call to discuss this. I could pick your mind for sometime to understand what would best help predict the next price.
# GridSearchCV: Get the hyperparameters that give us best results for the our current training data
# Neural Networks: Maybe we can ditch tree-based models altogether, and see if NNs give better performance.

In [36]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

ModuleNotFoundError: No module named 'pydot'